Import Libraries

In [25]:
import pandas as pd 
from tqdm import tqdm
import os
import torch
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
import PIL
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report

In [26]:
folder_path = 'Auto_ZMJ2N'

In [27]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, images_folder, transform = None):
        self.df = pd.read_csv(csv_path)
        self.images_folder = images_folder
        self.transform = transform

    def __len__(self):
        return len(self.df)
    def __getitem__(self, index):
        filename = self.df.loc[index]['filename']
        labels = self.df.iloc[index][1:].values.astype(float)
        image = PIL.Image.open(os.path.join(self.images_folder, filename))
        if self.transform is not None:
            image = self.transform(image)
        return image, torch.tensor(labels)

In [28]:
test_path = os.path.join(folder_path, 'test')
train_path = os.path.join(folder_path, 'train')
df = pd.read_csv(folder_path+'/classes.csv')
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
train_dataset = CustomDataset(folder_path+'/classes.csv', train_path, transform=transform)
test_dataset = datasets.ImageFolder(test_path,transform=transform)
print(test_dataset.classes)
# train_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.9, 0.1])

['OK', 'blobs', 'spaghetti', 'stringing']


In [30]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

num_of_classes = 4
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_of_classes)
model.to(device)

/home/jadhaiza/miniconda3/envs/ka/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jadhaiza/miniconda3/envs/ka/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [31]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

train_loader =  DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader =  DataLoader(test_dataset, batch_size=32, shuffle=False)

In [32]:
# Hyper parameters 

epochs = 10

In [33]:

for epoch in tqdm(range(epochs), desc="Training", unit="epoch"):
    running_loss = 0.0
    total = 0
    correct = 0
    train_loss_history = []
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        labels = torch.argmax(labels, dim=1)  
        correct += (predicted == labels).sum().item()
        accuracy = correct / total * 100
        
    print(f'epoch: {epoch + 1} loss: {running_loss / len(train_loader):.3f}, accuracy: {accuracy:.2f}%')
    running_loss = 0.0
    total = 0
    correct = 0

print('Finished Training')

Training:  10%|███████████▎                                                                                                     | 1/10 [00:06<00:59,  6.59s/epoch]

epoch: 1 loss: 1.029, accuracy: 58.61%


Training:  20%|██████████████████████▌                                                                                          | 2/10 [00:13<00:52,  6.58s/epoch]

epoch: 2 loss: 0.729, accuracy: 73.76%


Training:  30%|█████████████████████████████████▉                                                                               | 3/10 [00:19<00:46,  6.57s/epoch]

...

epoch: 8 loss: 0.473, accuracy: 84.16%


Training:  90%|█████████████████████████████████████████████████████████████████████████████████████████████████████▋           | 9/10 [00:59<00:06,  6.58s/epoch]

epoch: 9 loss: 0.459, accuracy: 82.52%


Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:05<00:00,  6.58s/epoch]

epoch: 10 loss: 0.443, accuracy: 84.36%
Finished Training


In [37]:
# Define a mapping for label alignment
label_mapping = {
    0: 0,  # Keep class 0 as is
    1: 3,  # Map class 1 to 3
    2: 1,  # Map class 2 to 1
    3: 2   # Map class 3 to 2
}

In [38]:
model.eval()

correct = 0
total = 0
all_labels = []
all_preds = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        # Apply the label mapping
        real = torch.tensor([label_mapping[label.item()] for label in labels], device=device)

        total += labels.size(0)
        all_labels.extend(real.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())
        correct += (predicted == real).sum().item()

accuracy = correct / total * 100
print(f'Test Accuracy: {accuracy:.2f}%') 

Test Accuracy: 89.04%


In [39]:
report = classification_report(all_labels, all_preds, target_names=['OK','Spaghetti','stringing','blobs'])
print(report)

              precision    recall  f1-score   support

          OK       0.92      0.92      0.92       123
   Spaghetti       1.00      0.88      0.94       100
   stringing       0.84      0.83      0.83        86
       blobs       0.75      0.96      0.84        47

    accuracy                           0.89       356
   macro avg       0.88      0.90      0.88       356
weighted avg       0.90      0.89      0.89       356

